In [1]:
import requests
from bs4 import BeautifulSoup
import web_scraper as ws
import json

In [2]:
header = {
    "User-Agents" : "Browser Details"
}
url = "https://huggingface.co/"

## To list unique tags inside a webpage url

In [3]:
resp = requests.get(url=url, headers=header)
soup = BeautifulSoup(resp.content, "html.parser")

title = soup.title.string if soup.title else "No-title"
if soup.body:
    print(list(set([x.name for x in soup.body.find_all()])))


['button', 'footer', 'header', 'li', 'article', 'path', 'a', 'h2', 'img', 'ul', 'input', 'main', 'h4', 'svg', 'nav', 'h3', 'div', 'time', 'hr', 'ellipse', 'span', 'script', 'h1', 'p']


## Extracting relavent links from an Website using LLM

In [4]:
system_prompt = """
Assume you are an expert in understanding an website and you have been provide with the complete urls that are present 
in the website and you need to extract the urls that are very much relavent to the website.
Share us the result in an Json format as below.
{
   "links": [
    {"type" : "about","url": "url_that_gives_information_about_website"},
    {"type" : "carrers" , "url" : "url_that_gives_information_about_carrers_inside_that_website"}
    ]
}
"""

In [5]:
def user_prompt_link(url,header):
    user_prompt = f"""
    As assumed your are the expert in understanding website page for the given website {url},
    extract the relavent information and share us the details in Json format and exclude website terms and service
    conditions.
    Passing the relavent title concatenated with URLS of the websites as follows:

    """
    wb_url = ws.fetch_website_url(url,header)
    return user_prompt + wb_url

In [6]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [7]:
from openai import OpenAI

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="Hardcode_api_key_for_ollama_local_model")

response_ollama = ollama.chat.completions.create(
    model="llama3.2",
    messages= [
        {"role":"system", "content": "system_prompt"},
        {"role":"user", "content":user_prompt_link(url,header)}
    ],
    response_format={"type":"json_object"}
)

resp = response_ollama.choices[0].message.content
print(resp)

{

  "web_pages": [
    {
      "title": "Hugging Face – The AI community building the future.",
      "links": ["https://huggingface.co"]
    },
    {
      "title": "Models",
      "links": ["https://huggingface.co/models"]
    },
    {
      "title": "Datasets",
      "links": ["https://huggingface.co/datasets"]
    },
    {
      "title": "Spaces",
      "links": ["https://huggingface.co/spaces"]
    },
    {
      "title": "Documentation",
      "links": [
        "https://huggingface.co/docs",
        "https://huggingface.co/docs/transformers"
      ]
    },
    {
      "title": "Enterprise",
      "links": ["https://huggingface.co/enterprise"]
    },
    {
      "title": "Pricing",
      "links": [
        "https://huggingface.co/pricing",
        "https://endpoints.huggingface.co"
      ]
    },
    {
      "title": "Join",
      "links": ["https://huggingface.co/join"]
    },
    {
      "title": "Inference models",
      "links": ["https://huggingface.co/inference/models"]
  

## Reading an website and creating a brocher based on its content

In [12]:
broucher_sys_promt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
"""

In [13]:
def broucher_user_prompt(url,header):
    user_prompt = f"""
    Your looking at the company with url f{'url'} and concatinated with website contents as below
    and need to assit in creating a short broucher with website landing page content details
    """
    wb_content = ws.webpage_content(url,header)
    return user_prompt + wb_content

In [14]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [17]:
from IPython.display import Markdown
from openai import OpenAI
resp = ''

ollama = OpenAI(base_url=OLLAMA_BASE_URL,api_key='HARDCODE_API_KEY')

stream = ollama.chat.completions.create(
    model='llama3.2',
    messages=[
        {'role':'system', 'content':broucher_sys_promt},
        {'role':'user', 'content':broucher_user_prompt(url,header)}
    ],
    stream=True
)

for chunk in stream:
    resp += chunk.choices[0].delta.content
    # display(Markdown(resp), display_id=True)
    # display(resp)
print(resp)

Based on the website content of Hugging Face, I've created a short brochure for prospective customers, investors, and recruits. Here's a draft:

**Welcome to Hugging Face: The AI Community Building the Future**

At Hugging Face, we're committed to building an open community that fosters collaboration, innovation, and progress in artificial intelligence (AI). Our platform is designed to bring together machine learning practitioners, researchers, and developers from around the world.

**Explore the Power of AI with Our Models, Datasets, and Applications**

We offer a vast library of 2M+ models, showcasing cutting-edge technology in various domains. From text-to-speech synthesis to image generation, our models can tackle some of the most complex challenges in AI research. Browse our trending models on this week:

* moonshotai/Kim-K2.5: A cutting-edge model for self-supervised learning
* Tongyi-MAI/Z-Image: An impressive model for object detection and segmentation

**Get Inspired by Our Ap